In [1]:
pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=fd8f92fae3f515d658c25b2983062ab2cc04232a9840dbbd311567f9a68f6fce
  Stored in directory: c:\users\fcanb\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\fcanb\\anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
TEST_SIZE = 0.5
RANDOM_STATE = 2018
BATCH_SIZE = 64
NO_EPOCHS = 20
NUM_CLASSES = 2
SAMPLE_SIZE = 20000
PATH = '/kaggle/input/dogs-vs-cats-redux-kernels-edition/'
TRAIN_FOLDER = './train/'
TEST_FOLDER =  './test/'
IMG_SIZE = 224

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.applications.mobilenet import MobileNet

import math

In [ ]:
def getModel(dropout=.25, learning_rate=0.001, augmentation=False):
  # 'frozen' base model
  base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
  base_model.trainable = False

  # input layer
  inputs = tf.keras.Input(shape=(299, 299, 3))
  x = inputs
  
  # pass results to a preprocessing layer
  x = tf.keras.applications.inception_v3.preprocess_input(x)
  
  # send results to the already trained model
  x = base_model(x, training=False)
  
  # send results to pooling layer
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  
  # send to fully connected layer
  x = tf.keras.layers.Dense(256,activation='relu')(x)
  
  # send to dropout layer
  x = tf.keras.layers.Dropout(dropout)(x)
  
  # send to batch normalization layer
  x = tf.keras.layers.BatchNormalization()(x)
   
  # send to output dense layer with SoftMax activation function
  outputs = tf.keras.layers.Dense(37,activation='softmax')(x)
  
  # create a model with these layers
  model = tf.keras.Model(inputs, outputs)
  
  # show some information
  model.summary()

  # compile model with Adam optimizer
  # and chosing accuracy as the metric to evaluate the model performance during training
  model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])

  return model


In [ ]:
import random
import cv2
import numpy as np

# setting image pixel side size
IMG_SIZE = 299

def getXy():
  # function that returns the number correspondent to the breed of 
  # the animal in the image, given the image name
  get_class_no = lambda name : info_by_breed[get_breed(name)]['globalid']
  
  # all image tensors will be stored here after resizing
  training_data = []
  
  for img in all_imgs:
    path = os.path.join(IMGS_PATH, img)
  
    # this is a trick to make the image be opened in RGB format, which is not the default
    img_array = cv2.imread(path)[...,::-1] 

    # here the images are rezise
    img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
  
    # get the ID of the image class
    class_no = get_class_no(img)
  
    training_data.append([img_array, class_no])
    
  # data should be in random order to improve performance
  random.shuffle(training_data)
  
  # separating data from list
  training = list(zip(*training_data))
  X = training[0]
  y = training[1]
  
  # transforming X to an np.array and resizing
  X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
  y = np.array(y)
  return X, y

In [ ]:

# one more import
from sklearn.model_selection import train_test_split

# stratified split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [ ]:

# create dictionary to save data
lr = {}

# lambda function to be passed in test_params function
lr_model_func = lambda x : getModel(learning_rate=x)

# we are testing only with 0.01 and 0.001
for lr_val in [0.01, 0.001]:
  # for each value, call function and save data in dictionary
  lr[lr_val] = test_params(lr_val, lr_model_func)